In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np

In [3]:
from nflows.flows import MaskedAutoregressiveFlow

from counterfactuals.datasets.heloc import HelocDataset
from counterfactuals.datasets.moons import MoonsDataset
from counterfactuals.datasets.law import LawDataset
from counterfactuals.datasets.compas import CompasDataset

from counterfactuals.optimizers.approach_gen_disc import ApproachGenDisc

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

from counterfactuals.metrics.metrics import (
    perc_valid_cf,
    perc_valid_actionable_cf,
    continuous_distance,
    categorical_distance,
    distance_l2_jaccard,
    distance_mad_hamming,
    plausibility,
    delta_proba
)

/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = CompasDataset(file_path="../data/origin/compas_two_years.csv")
train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)

In [5]:
# clf = LogisticRegression()
clf = MLPClassifier((128, 64), max_iter=100)
clf.fit(dataset.X_train, dataset.y_train)
y_pred_train = clf.predict(dataset.X_train)
y_pred_test = clf.predict(dataset.X_test)
print(classification_report(dataset.y_test, y_pred_test, output_dict=False))

              precision    recall  f1-score   support

         0.0       0.79      0.77      0.78       423
         1.0       0.78      0.79      0.78       419

    accuracy                           0.78       842
   macro avg       0.78      0.78      0.78       842
weighted avg       0.78      0.78      0.78       842



/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


In [6]:
flow = MaskedAutoregressiveFlow(features=dataset.X_test.shape[1], hidden_features=4, context_features=None)
cf = ApproachGenDisc(gen_model=flow, disc_model=clf)

In [7]:
cf.train_model(
    train_loader=train_dataloader,
    test_loader=test_dataloader,
    epochs=200,
    verbose=True
)

Epochs:   2%|▏         | 3/200 [00:00<00:17, 11.02it/s]

Epoch 0, Train: 43.587529828471524, test: 40.9719249180385


Epochs:   6%|▋         | 13/200 [00:01<00:15, 11.78it/s]

Epoch 10, Train: -0.2981909133734242, test: -0.4081937564270837


Epochs:  12%|█▏        | 23/200 [00:01<00:15, 11.68it/s]

...

Epochs:  92%|█████████▏| 183/200 [00:15<00:01, 12.19it/s]

Epoch 180, Train: -20.055115607477003, test: -21.225250925336564


Epochs:  96%|█████████▋| 193/200 [00:15<00:00, 12.17it/s]

Epoch 190, Train: -21.634810909148186, test: -21.794276646205358


Epochs: 100%|██████████| 200/200 [00:16<00:00, 12.11it/s]


In [8]:
cf.test_model(test_loader=test_dataloader)

              precision    recall  f1-score   support

         0.0       0.50      1.00      0.67       423
         1.0       0.00      0.00      0.00       419

    accuracy                           0.50       842
   macro avg       0.25      0.50      0.33       842
weighted avg       0.25      0.50      0.34       842



/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ofurman/Study/counterfactuals/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [9]:
Xs_cfs = cf.generate_counterfactuals(Xs=dataset.X_test[:100], ys=dataset.y_test[:100], num_epochs=100, lr=0.005, alpha=20, beta=0.01)
Xs_cfs = torch.concat(Xs_cfs).detach()

0it [00:00, ?it/s]

100it [00:25,  3.85it/s]


In [11]:
y_cfs_pred_mlpc = clf.predict(Xs_cfs)
y_orig_pred_mlpc = clf.predict(dataset.X_test[:100])

In [12]:
X = dataset.X_test[:100]
ys_cfs_pred = clf.predict(Xs_cfs)
ys_orig_pred = clf.predict(dataset.X_test[:100])
ys_orig = dataset.y_test[:100].flatten()

{
    "valid_cf": perc_valid_cf(ys_orig_pred, y_cf=ys_cfs_pred),
    "valid_cf_for_orig_data": perc_valid_cf(ys_orig, y_cf=ys_cfs_pred),
    # "perc_valid_actionable_cf": perc_valid_actionable_cf(X=dataset.X_test[:100], X_cf=Xs_cfs, y=ys_orig_pred, y_cf=ys_cfs_pred, actionable_features=[1,2]),
    "continuous_distance": continuous_distance(X=X, X_cf=Xs_cfs, continuous_features=dataset.continuous_columns, metric='mad', X_all=dataset.X_test),
    "categorical_distance": categorical_distance(X=X, X_cf=Xs_cfs, categorical_features=dataset.categorical_columns, metric='jaccard', agg='mean'),
    "distance_l2_jaccard": distance_l2_jaccard(X=X, X_cf=Xs_cfs, continuous_features=dataset.continuous_columns, categorical_features=dataset.categorical_columns),
    "distance_mad_hamming": distance_mad_hamming(X=X, X_cf=Xs_cfs,
                                                 continuous_features=dataset.continuous_columns, categorical_features=dataset.categorical_columns, X_all=X, agg='mean'),
    "plausibility": plausibility(
        X, Xs_cfs, ys_orig,
        continuous_features_all=dataset.continuous_columns,
        categorical_features_all=dataset.categorical_columns,
        X_train=dataset.X_train,
        ratio_cont=None
    ),
    "delta_probability": delta_proba(X, Xs_cfs, classifier=clf),
    "log_density": np.mean(cf.predict_model(test_dataloader))
}

{'valid_cf': 0.02,
 'valid_cf_for_orig_data': 0.22,
 'continuous_distance': 10.4251913975653,
 'categorical_distance': 0.48,
 'distance_l2_jaccard': 0.27139890054267696,
 'distance_mad_hamming': 5.112228220476975,
 'plausibility': 7.521560593275668,
 'delta_probability': 0.020853292,
 'log_density': 21.179031}